## 卷积层
卷积相当于全连接的进化版，将w从一维变成多维度

In [47]:
import torch
from torch import nn
from d2l import torch as d2l

In [48]:
print(torch.cuda.is_available())
device = torch.device("cuda")

True


In [49]:
def corr2d(X, K):
    '''计算二维互相关运算'''

    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))

    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()

    return Y

In [50]:
X = torch.arange(0, 9).reshape(3, 3)
K = torch.arange(0, 4).reshape(2, 2)
print(corr2d(X, K))

tensor([[19., 25.],
        [37., 43.]])


In [51]:
# 实现二维卷积层

from turtle import forward


class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1)) # 使用广播机制，所以bias大小是1

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [52]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
print(X)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


In [53]:
K = torch.tensor([[1.0, -1.0]])

In [54]:
Y = corr2d(X, K)
print(Y)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [55]:
print(corr2d(X.T, K))

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [57]:
# 学习由X生成Y的卷积核

from numpy import divide


conv2d = nn.Conv2d(1, 1, (1, 2), bias= False).to(device)

# (N, C, H, W) N:输入图片数量；C:图像的通道数；H：图像高度；W:图像宽度
X = X.reshape((1, 1, 6, 8)).to(device)
Y = Y.reshape((1, 1, 6, 7)).to(device)

epochs = 10
for epoch in range(epochs):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (epoch + 1) % 2 == 0:
        print(f"batch {epoch + 1}, loss {l.sum():.3f}")

batch 2, loss 18.362
batch 4, loss 6.154
batch 6, loss 2.291
batch 8, loss 0.900
batch 10, loss 0.362


In [ ]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0549, -0.9209]])